In [2]:
# ╔═══════════════════════════════════════════════════════════════════════════╗
# ║   SHA-256 ULTRA COLLISION HUNTER v4.1 - FIXED                            ║
# ║   Multi-Core + Vectorized + Maximum Speed                                 ║
# ║   Author: Kaoru Aguilera Katayama                                        ║
# ╚═══════════════════════════════════════════════════════════════════════════╝

import hashlib
import numpy as np
from collections import defaultdict
from typing import Dict, List, Tuple
import time
import multiprocessing as mp
import warnings
warnings.filterwarnings('ignore')

NUM_CORES = mp.cpu_count()

# Fixed: Use regular Python list instead of numpy array to avoid uint8 overflow
POPCOUNT_TABLE = [bin(i).count('1') for i in range(256)]

# ═══════════════════════════════════════════════════════════════════════════
# BANNER
# ═══════════════════════════════════════════════════════════════════════════

def show_banner():
    print(f"""
╔═══════════════════════════════════════════════════════════════════════════╗
║                                                                           ║
║   ███████╗██╗  ██╗ █████╗       ██████╗ ███████╗ ██████╗                  ║
║   ██╔════╝██║  ██║██╔══██╗      ╚════██╗██╔════╝██╔════╝                  ║
║   ███████╗███████║███████║█████╗ █████╔╝███████╗███████╗                  ║
║   ╚════██║██╔══██║██╔══██║╚════╝██╔═══╝ ╚════██║██╔═══██╗                 ║
║   ███████║██║  ██║██║  ██║      ███████╗███████║╚██████╔╝                 ║
║   ╚══════╝╚═╝  ╚═╝╚═╝  ╚═╝      ╚══════╝╚══════╝ ╚═════╝                  ║
║                                                                           ║
║              ULTRA COLLISION HUNTER v4.1                                  ║
║                                                                           ║
║   🔥 {NUM_CORES} CPU Cores Detected                                            ║
║   🎯 Current Record: 178/256 bits                                         ║
║   🚀 Target: 256/256 bits                                                 ║
║                                                                           ║
║   Author: Kaoru Aguilera Katayama                                        ║
║                                                                           ║
╚═══════════════════════════════════════════════════════════════════════════╝
    """)

# ═══════════════════════════════════════════════════════════════════════════
# ULTRA-FAST CORE FUNCTIONS
# ═══════════════════════════════════════════════════════════════════════════

def sha256_bytes(msg: int) -> bytes:
    return hashlib.sha256(str(msg).encode()).digest()

def sha256_hex(msg: int) -> str:
    return hashlib.sha256(str(msg).encode()).hexdigest()

def count_equal_bits(h1_bytes: bytes, h2_bytes: bytes) -> int:
    """Count equal bits between two hash byte arrays"""
    different = 0
    for a, b in zip(h1_bytes, h2_bytes):
        different += POPCOUNT_TABLE[a ^ b]
    return 256 - different

def count_equal_bits_hex(h1: str, h2: str) -> int:
    """Count equal bits between two hex hash strings"""
    xor = int(h1, 16) ^ int(h2, 16)
    return 256 - bin(xor).count('1')

# ═══════════════════════════════════════════════════════════════════════════
# PARALLEL WORKERS
# ═══════════════════════════════════════════════════════════════════════════

def worker_random_search(args):
    """Massive random search"""
    n_pairs, msg_range, seed = args

    import random
    random.seed(seed)

    best = (0, 0, 0)

    for _ in range(n_pairs):
        m1 = random.randint(1, msg_range - 1)
        m2 = random.randint(1, msg_range - 1)

        if m1 != m2:
            h1 = sha256_bytes(m1)
            h2 = sha256_bytes(m2)
            bits = count_equal_bits(h1, h2)

            if bits > best[2]:
                best = (m1, m2, bits)

    return best

def worker_neighborhood(args):
    """Neighborhood search"""
    center1, center2, radius, samples, seed = args

    import random
    random.seed(seed)

    best = (0, 0, 0)

    points1 = [max(1, center1 + random.randint(-radius, radius)) for _ in range(samples)]
    points2 = [max(1, center2 + random.randint(-radius, radius)) for _ in range(samples)]

    hashes1 = {p: sha256_bytes(p) for p in set(points1)}
    hashes2 = {p: sha256_bytes(p) for p in set(points2)}

    for p1, h1 in hashes1.items():
        for p2, h2 in hashes2.items():
            if p1 != p2:
                bits = count_equal_bits(h1, h2)
                if bits > best[2]:
                    best = (p1, p2, bits)

    return best

def worker_xor_differential(args):
    """XOR differential analysis"""
    start, end, deltas = args

    best = (0, 0, 0)

    for m in range(start, end):
        h1 = sha256_bytes(m)

        for delta in deltas:
            m2 = m ^ delta
            if m2 > 0 and m2 != m:
                h2 = sha256_bytes(m2)
                bits = count_equal_bits(h1, h2)

                if bits > best[2]:
                    best = (m, m2, bits)

    return best

def worker_prefix_batch(args):
    """Process prefix batch"""
    start, end, prefix_len = args

    prefixes = defaultdict(list)

    for i in range(start, end):
        h = sha256_hex(i)
        prefixes[h[:prefix_len]].append((i, h))

    best = (0, 0, 0)

    for prefix, group in prefixes.items():
        if len(group) >= 2:
            for i in range(len(group)):
                for j in range(i+1, len(group)):
                    m1, h1 = group[i]
                    m2, h2 = group[j]
                    bits = count_equal_bits_hex(h1, h2)

                    if bits > best[2]:
                        best = (m1, m2, bits)

    return best

def worker_birthday(args):
    """Birthday attack batch"""
    batch_start, batch_size, prefix_bytes = args

    prefixes = defaultdict(list)

    for i in range(batch_size):
        m = batch_start + i
        h = sha256_bytes(m)
        prefix = h[:prefix_bytes]
        prefixes[prefix].append((m, h))

    best = (0, 0, 0)

    for prefix, group in prefixes.items():
        if len(group) >= 2:
            for i in range(len(group)):
                for j in range(i+1, len(group)):
                    m1, h1 = group[i]
                    m2, h2 = group[j]
                    bits = count_equal_bits(h1, h2)

                    if bits > best[2]:
                        best = (m1, m2, bits)

    return best

def worker_sequential_diff(args):
    """Check sequential numbers with small differences"""
    start, end, max_diff = args

    best = (0, 0, 0)

    for m1 in range(start, end):
        h1 = sha256_bytes(m1)

        for diff in range(1, max_diff + 1):
            m2 = m1 + diff
            h2 = sha256_bytes(m2)
            bits = count_equal_bits(h1, h2)

            if bits > best[2]:
                best = (m1, m2, bits)

    return best

# ═══════════════════════════════════════════════════════════════════════════
# MAIN HUNTER
# ═══════════════════════════════════════════════════════════════════════════

class UltraHunter:

    def __init__(self):
        self.best_bits = 178
        self.best_m1 = 18521
        self.best_m2 = 30321
        self.best_h1 = sha256_hex(18521)
        self.best_h2 = sha256_hex(30321)
        self.total_ops = 0
        self.improvements = 0

    def update(self, m1, m2, bits):
        if bits > self.best_bits and m1 != m2:
            self.best_bits = bits
            self.best_m1 = m1
            self.best_m2 = m2
            self.best_h1 = sha256_hex(m1)
            self.best_h2 = sha256_hex(m2)
            self.improvements += 1
            return True
        return False

    def phase_random(self, n_total: int = 50_000_000, msg_range: int = 100_000_000):
        """Phase 1: Massive random search"""

        print(f"\n{'='*70}")
        print(f"🎲 PHASE 1: MASSIVE RANDOM SEARCH")
        print(f"{'='*70}")
        print(f"   Pairs: {n_total:,} | Range: 1-{msg_range:,} | Cores: {NUM_CORES}")

        pairs_per_worker = n_total // NUM_CORES

        with mp.Pool(NUM_CORES) as pool:
            args = [(pairs_per_worker, msg_range, i*12345) for i in range(NUM_CORES)]
            results = pool.map(worker_random_search, args)

        for m1, m2, bits in results:
            if self.update(m1, m2, bits):
                print(f"   ✨ NEW! {bits}/256 bits - M1={m1}, M2={m2}")

        self.total_ops += n_total
        print(f"   → Best: {self.best_bits}/256 bits")

    def phase_prefix(self, N: int = 5_000_000):
        """Phase 2: Prefix attack"""

        print(f"\n{'='*70}")
        print(f"🔤 PHASE 2: PREFIX ATTACK")
        print(f"{'='*70}")

        for prefix_len in [7, 6, 5]:
            print(f"\n   Prefix length: {prefix_len} characters...")

            batch_size = N // NUM_CORES

            with mp.Pool(NUM_CORES) as pool:
                args = [(i*batch_size + 1, (i+1)*batch_size + 1, prefix_len) for i in range(NUM_CORES)]
                results = pool.map(worker_prefix_batch, args)

            for m1, m2, bits in results:
                if self.update(m1, m2, bits):
                    print(f"   ✨ NEW! {bits}/256 bits")

            self.total_ops += N

        print(f"   → Best: {self.best_bits}/256 bits")

    def phase_differential(self, N: int = 10_000_000):
        """Phase 3: Differential analysis"""

        print(f"\n{'='*70}")
        print(f"⊕ PHASE 3: XOR DIFFERENTIAL ANALYSIS")
        print(f"{'='*70}")

        deltas = [1, 2, 3, 4, 5, 6, 7, 8, 15, 16, 31, 32, 37, 63, 64,
                  127, 128, 255, 256, 511, 512, 1023, 1024, 2047, 4096]

        batch_size = N // NUM_CORES

        with mp.Pool(NUM_CORES) as pool:
            args = [(i*batch_size + 1, (i+1)*batch_size + 1, deltas) for i in range(NUM_CORES)]
            results = pool.map(worker_xor_differential, args)

        for m1, m2, bits in results:
            if self.update(m1, m2, bits):
                print(f"   ✨ NEW! {bits}/256 bits - Δ={m1^m2}")

        self.total_ops += N * len(deltas)
        print(f"   → Best: {self.best_bits}/256 bits")

    def phase_sequential(self, N: int = 5_000_000, max_diff: int = 100):
        """Phase 3.5: Sequential difference search"""

        print(f"\n{'='*70}")
        print(f"📈 PHASE 3.5: SEQUENTIAL DIFFERENCE SEARCH")
        print(f"{'='*70}")
        print(f"   Range: 1-{N:,} | Max diff: {max_diff}")

        batch_size = N // NUM_CORES

        with mp.Pool(NUM_CORES) as pool:
            args = [(i*batch_size + 1, (i+1)*batch_size + 1, max_diff) for i in range(NUM_CORES)]
            results = pool.map(worker_sequential_diff, args)

        for m1, m2, bits in results:
            if self.update(m1, m2, bits):
                print(f"   ✨ NEW! {bits}/256 bits - M1={m1}, M2={m2}, diff={m2-m1}")

        self.total_ops += N * max_diff
        print(f"   → Best: {self.best_bits}/256 bits")

    def phase_neighborhood(self, iterations: int = 50):
        """Phase 4: Neighborhood refinement"""

        print(f"\n{'='*70}")
        print(f"🎯 PHASE 4: NEIGHBORHOOD REFINEMENT")
        print(f"{'='*70}")

        for i in range(iterations):
            radius = max(100, 10000 - i*200)
            samples = 500

            with mp.Pool(NUM_CORES) as pool:
                args = [(self.best_m1, self.best_m2, radius, samples, i*1000 + j)
                        for j in range(NUM_CORES)]
                results = pool.map(worker_neighborhood, args)

            for m1, m2, bits in results:
                if self.update(m1, m2, bits):
                    print(f"   ✨ Iter {i}: NEW! {bits}/256 bits")

            self.total_ops += NUM_CORES * samples * samples

            if i % 10 == 0:
                print(f"   Iter {i}/{iterations} - Radius: {radius} - Best: {self.best_bits}/256")

    def phase_birthday(self, batches: int = 200, batch_size: int = 500_000):
        """Phase 5: Birthday attack"""

        print(f"\n{'='*70}")
        print(f"🎂 PHASE 5: BIRTHDAY ATTACK")
        print(f"{'='*70}")
        print(f"   Batches: {batches} | Size: {batch_size:,} | Total: {batches*batch_size:,}")

        import random

        for prefix_bytes in [4, 3]:
            print(f"\n   Using {prefix_bytes} prefix bytes...")

            batches_per_iter = NUM_CORES

            for batch_num in range(0, batches, batches_per_iter):
                with mp.Pool(NUM_CORES) as pool:
                    args = [(random.randint(1, 1_000_000_000) + i*batch_size,
                             batch_size, prefix_bytes)
                            for i in range(batches_per_iter)]
                    results = pool.map(worker_birthday, args)

                for m1, m2, bits in results:
                    if self.update(m1, m2, bits):
                        print(f"   ✨ NEW! {bits}/256 bits")

                self.total_ops += batches_per_iter * batch_size

                if batch_num % 50 == 0:
                    print(f"   Batch {batch_num}/{batches} - Best: {self.best_bits}/256")

    def phase_intensive(self, radius: int = 500):
        """Phase 6: Final intensive search"""

        print(f"\n{'='*70}")
        print(f"🔬 PHASE 6: FINAL INTENSIVE SEARCH")
        print(f"{'='*70}")
        print(f"   Center: ({self.best_m1}, {self.best_m2}) | Radius: ±{radius}")

        # Generate all points
        range1 = list(range(max(1, self.best_m1 - radius), self.best_m1 + radius + 1))
        range2 = list(range(max(1, self.best_m2 - radius), self.best_m2 + radius + 1))

        all_points = list(set(range1 + range2))
        print(f"   Points: {len(all_points):,}")

        # Pre-compute hashes
        print("   Computing hashes...")
        hashes = {p: sha256_bytes(p) for p in all_points}

        # Compare all pairs
        print("   Comparing pairs...")
        total_pairs = len(all_points) * (len(all_points) - 1) // 2
        checked = 0

        for i in range(len(all_points)):
            for j in range(i+1, len(all_points)):
                p1, p2 = all_points[i], all_points[j]
                bits = count_equal_bits(hashes[p1], hashes[p2])

                if self.update(p1, p2, bits):
                    print(f"   ✨ NEW! {bits}/256 bits")

                checked += 1

            if i % 200 == 0 and i > 0:
                pct = 100 * checked / total_pairs
                print(f"   Progress: {checked:,}/{total_pairs:,} ({pct:.1f}%)")

        self.total_ops += total_pairs
        print(f"   → Best: {self.best_bits}/256 bits")

    def run(self, mode: str = "normal"):
        """Run all phases"""

        configs = {
            "quick": {
                "random": 10_000_000,
                "prefix": 1_000_000,
                "diff": 2_000_000,
                "sequential": 1_000_000,
                "neighborhood": 20,
                "birthday": 50,
                "intensive": 200
            },
            "normal": {
                "random": 50_000_000,
                "prefix": 5_000_000,
                "diff": 10_000_000,
                "sequential": 5_000_000,
                "neighborhood": 50,
                "birthday": 100,
                "intensive": 500
            },
            "intensive": {
                "random": 100_000_000,
                "prefix": 10_000_000,
                "diff": 20_000_000,
                "sequential": 10_000_000,
                "neighborhood": 100,
                "birthday": 200,
                "intensive": 1000
            },
            "extreme": {
                "random": 500_000_000,
                "prefix": 50_000_000,
                "diff": 50_000_000,
                "sequential": 20_000_000,
                "neighborhood": 200,
                "birthday": 500,
                "intensive": 2000
            }
        }

        cfg = configs.get(mode, configs["normal"])
        start = time.time()

        print(f"\n{'='*70}")
        print(f"🚀 STARTING - Mode: {mode.upper()}")
        print(f"{'='*70}")

        self.phase_random(cfg["random"])
        self.phase_prefix(cfg["prefix"])
        self.phase_differential(cfg["diff"])
        self.phase_sequential(cfg["sequential"])
        self.phase_neighborhood(cfg["neighborhood"])
        self.phase_birthday(cfg["birthday"])
        self.phase_intensive(cfg["intensive"])

        elapsed = time.time() - start
        return elapsed

    def show_results(self, elapsed: float):
        """Show results"""

        pct = self.best_bits / 256 * 100
        bar = "█" * int(pct // 5) + "░" * (20 - int(pct // 5))

        status = "🎉 NEW RECORD!" if self.best_bits > 178 else "📊 Result"

        print(f"""

{'='*70}
🏆 FINAL RESULTS
{'='*70}

╔═══════════════════════════════════════════════════════════════════════════╗
║                                                                           ║
║   {status:^67} ║
║                                                                           ║
║   MATCHING BITS: {self.best_bits}/256 ({pct:.2f}%)
║   [{bar}]
║                                                                           ║
║   M1 = {self.best_m1}
║   M2 = {self.best_m2}
║                                                                           ║
║   SHA-256(M1) = {self.best_h1}
║   SHA-256(M2) = {self.best_h2}
║                                                                           ║
║   ⏱️  Time: {elapsed:.1f} seconds
║   🔢 Operations: {self.total_ops:,}
║   📈 Improvements: {self.improvements}
║                                                                           ║
╚═══════════════════════════════════════════════════════════════════════════╝
        """)

        # Bit map
        print("📊 BIT MAP (█=match, ░=diff):\n")
        b1 = bin(int(self.best_h1, 16))[2:].zfill(256)
        b2 = bin(int(self.best_h2, 16))[2:].zfill(256)

        for row in range(4):
            line = "   "
            for col in range(64):
                idx = row * 64 + col
                line += "█" if b1[idx] == b2[idx] else "░"
            print(line)

        # Verification
        verify_h1 = hashlib.sha256(str(self.best_m1).encode()).hexdigest()
        verify_h2 = hashlib.sha256(str(self.best_m2).encode()).hexdigest()
        verify_bits = count_equal_bits_hex(verify_h1, verify_h2)

        print(f"""
✅ VERIFICATION:
   M1 ≠ M2: {'YES ✓' if self.best_m1 != self.best_m2 else 'NO'}
   SHA-256 (64 rounds): YES ✓
   Verified bits: {verify_bits}/256 {'✓' if verify_bits == self.best_bits else '✗'}
        """)

# ═══════════════════════════════════════════════════════════════════════════
# MAIN
# ═══════════════════════════════════════════════════════════════════════════

def main():
    show_banner()

    print(f"""
{'='*70}
⚙️  CONFIGURATION
{'='*70}

   1. Quick      (~5 min)     - Fast test
   2. Normal     (~15-30 min) - Standard search
   3. Intensive  (~1 hour)    - Deep search
   4. Extreme    (~3+ hours)  - Maximum effort
    """)

    try:
        choice = input("   Mode [1-4, default=2]: ").strip() or "2"
    except:
        choice = "2"

    modes = {"1": "quick", "2": "normal", "3": "intensive", "4": "extreme"}
    mode = modes.get(choice, "normal")

    print(f"\n   → Mode: {mode.upper()}")
    input("   Press ENTER to start...")

    hunter = UltraHunter()
    elapsed = hunter.run(mode)
    hunter.show_results(elapsed)

    print(f"""
{'='*70}
   ✅ COMPLETED
   Author: Kaoru Aguilera Katayama
   SHA-256 Ultra Collision Hunter v4.1
{'='*70}
    """)

if __name__ == "__main__":
    mp.freeze_support()

    try:
        main()
    except KeyboardInterrupt:
        print("\n\n   Interrupted. See you! 👋")
    except Exception as e:
        print(f"\n   Error: {e}")
        import traceback
        traceback.print_exc()


╔═══════════════════════════════════════════════════════════════════════════╗
║                                                                           ║
║   ███████╗██╗  ██╗ █████╗       ██████╗ ███████╗ ██████╗                  ║
║   ██╔════╝██║  ██║██╔══██╗      ╚════██╗██╔════╝██╔════╝                  ║
║   ███████╗███████║███████║█████╗ █████╔╝███████╗███████╗                  ║
║   ╚════██║██╔══██║██╔══██║╚════╝██╔═══╝ ╚════██║██╔═══██╗                 ║
║   ███████║██║  ██║██║  ██║      ███████╗███████║╚██████╔╝                 ║
║   ╚══════╝╚═╝  ╚═╝╚═╝  ╚═╝      ╚══════╝╚══════╝ ╚═════╝                  ║
║                                                                           ║
║              ULTRA COLLISION HUNTER v4.1                                  ║
║                                                                           ║
║   🔥 2 CPU Cores Detected                                            ║
║   🎯 Current Record: 178/256 bits                                   

Process ForkPoolWorker-3:




   Interrupted. See you! 👋


In [3]:
# ╔═══════════════════════════════════════════════════════════════════════════╗
# ║                                                                           ║
# ║   SHA-256 COLLISION FINDER - GPU ACCELERATED                             ║
# ║   Dimensional Space Method + CUDA/Multiprocessing                         ║
# ║                                                                           ║
# ║   Author: Kaoru Aguilera Katayama                                        ║
# ║                                                                           ║
# ║   RECORD: 174/256 bits (67.97%) - M1=88514, M2=88551                     ║
# ║                                                                           ║
# ╚═══════════════════════════════════════════════════════════════════════════╝

import hashlib
import numpy as np
from collections import defaultdict
import time
import warnings
import multiprocessing as mp
from typing import List, Tuple, Dict
import struct

warnings.filterwarnings('ignore')

# ═══════════════════════════════════════════════════════════════════════════
# GPU DETECTION AND SETUP
# ═══════════════════════════════════════════════════════════════════════════

GPU_AVAILABLE = False
CUDA_AVAILABLE = False
BACKEND = "CPU"

try:
    from numba import cuda, uint32, uint8
    import numba
    if cuda.is_available():
        CUDA_AVAILABLE = True
        GPU_AVAILABLE = True
        BACKEND = "CUDA"
        print("✅ CUDA GPU detected!")
except ImportError:
    pass

if not GPU_AVAILABLE:
    try:
        import cupy as cp
        GPU_AVAILABLE = True
        BACKEND = "CuPy"
        print("✅ CuPy GPU detected!")
    except ImportError:
        pass

NUM_CORES = mp.cpu_count()

# ═══════════════════════════════════════════════════════════════════════════
# SHA-256 CONSTANTS
# ═══════════════════════════════════════════════════════════════════════════

SHA256_K = np.array([
    0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5,
    0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
    0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3,
    0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
    0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc,
    0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
    0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7,
    0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
    0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13,
    0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
    0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3,
    0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
    0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5,
    0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
    0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208,
    0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
], dtype=np.uint32)

SHA256_H = np.array([
    0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
    0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19
], dtype=np.uint32)

POPCOUNT_TABLE = [bin(i).count('1') for i in range(256)]

# ═══════════════════════════════════════════════════════════════════════════
# BANNER
# ═══════════════════════════════════════════════════════════════════════════

def show_banner():
    print(f"""
╔═══════════════════════════════════════════════════════════════════════════╗
║                                                                           ║
║   ███████╗██╗  ██╗ █████╗       ██████╗ ███████╗ ██████╗                  ║
║   ██╔════╝██║  ██║██╔══██╗      ╚════██╗██╔════╝██╔════╝                  ║
║   ███████╗███████║███████║█████╗ █████╔╝███████╗███████╗                  ║
║   ╚════██║██╔══██║██╔══██║╚════╝██╔═══╝ ╚════██║██╔═══██╗                 ║
║   ███████║██║  ██║██║  ██║      ███████╗███████║╚██████╔╝                 ║
║   ╚══════╝╚═╝  ╚═╝╚═╝  ╚═╝      ╚══════╝╚══════╝ ╚═════╝                  ║
║                                                                           ║
║              COLLISION FINDER - GPU ACCELERATED                           ║
║                  Dimensional Space Method                                 ║
║                                                                           ║
║   🔥 Backend: {BACKEND:<10} | Cores: {NUM_CORES}                              ║
║   🎯 Record: 174/256 bits (67.97%)                                        ║
║   🚀 Target: 256/256 bits                                                 ║
║                                                                           ║
║   Author: Kaoru Aguilera Katayama                                        ║
║                                                                           ║
╚═══════════════════════════════════════════════════════════════════════════╝
    """)

def show_theory():
    print("""
═══════════════════════════════════════════════════════════════════════════
                    DIMENSIONAL SPACE METHOD - THEORY
═══════════════════════════════════════════════════════════════════════════

   📐 X-Axis: SHA(1), SHA(2), SHA(3)...  (hash values normalized)
   📐 Y-Axis: 1, 2, 3, 4, 5...           (message indices)
   📐 Z-Axis: Collisions                  (meeting point)

   LINEAR FUNCTION: Y = m·X + b
   COLLISION POINT: X = Y = Z → X = b/(1-m)

═══════════════════════════════════════════════════════════════════════════
    """)

# ═══════════════════════════════════════════════════════════════════════════
# CPU CORE FUNCTIONS
# ═══════════════════════════════════════════════════════════════════════════

def sha256_int(msg):
    return int(hashlib.sha256(str(msg).encode()).hexdigest(), 16)

def sha256_hex(msg):
    return hashlib.sha256(str(msg).encode()).hexdigest()

def sha256_bytes(msg):
    return hashlib.sha256(str(msg).encode()).digest()

def count_equal_bits(h1, h2):
    if isinstance(h1, str):
        xor = int(h1, 16) ^ int(h2, 16)
    else:
        xor = int.from_bytes(h1, 'big') ^ int.from_bytes(h2, 'big')
    return 256 - bin(xor).count('1')

def count_equal_bits_bytes(h1_bytes, h2_bytes):
    different = 0
    for a, b in zip(h1_bytes, h2_bytes):
        different += POPCOUNT_TABLE[a ^ b]
    return 256 - different

# ═══════════════════════════════════════════════════════════════════════════
# CUDA GPU IMPLEMENTATION (if available)
# ═══════════════════════════════════════════════════════════════════════════

if CUDA_AVAILABLE:

    @cuda.jit(device=True)
    def cuda_rotr(x, n):
        return ((x >> n) | (x << (32 - n))) & 0xFFFFFFFF

    @cuda.jit(device=True)
    def cuda_ch(x, y, z):
        return (x & y) ^ ((~x) & z)

    @cuda.jit(device=True)
    def cuda_maj(x, y, z):
        return (x & y) ^ (x & z) ^ (y & z)

    @cuda.jit(device=True)
    def cuda_sigma0(x):
        return cuda_rotr(x, 2) ^ cuda_rotr(x, 13) ^ cuda_rotr(x, 22)

    @cuda.jit(device=True)
    def cuda_sigma1(x):
        return cuda_rotr(x, 6) ^ cuda_rotr(x, 11) ^ cuda_rotr(x, 25)

    @cuda.jit(device=True)
    def cuda_gamma0(x):
        return cuda_rotr(x, 7) ^ cuda_rotr(x, 18) ^ (x >> 3)

    @cuda.jit(device=True)
    def cuda_gamma1(x):
        return cuda_rotr(x, 17) ^ cuda_rotr(x, 19) ^ (x >> 10)

    @cuda.jit
    def cuda_sha256_batch(messages, hashes_out, K):
        """Compute SHA-256 for a batch of integer messages on GPU"""
        idx = cuda.grid(1)

        if idx >= messages.shape[0]:
            return

        msg = messages[idx]

        # Convert integer to bytes (simple representation)
        msg_bytes = cuda.local.array(64, dtype=uint8)
        for i in range(64):
            msg_bytes[i] = 0

        # Simple integer to string conversion
        temp = msg
        length = 0
        temp_arr = cuda.local.array(20, dtype=uint8)

        if temp == 0:
            temp_arr[0] = 48  # '0'
            length = 1
        else:
            while temp > 0:
                temp_arr[length] = 48 + (temp % 10)
                temp = temp // 10
                length += 1

        # Reverse
        for i in range(length):
            msg_bytes[i] = temp_arr[length - 1 - i]

        # Padding
        msg_bytes[length] = 0x80
        bit_len = length * 8
        msg_bytes[62] = (bit_len >> 8) & 0xFF
        msg_bytes[63] = bit_len & 0xFF

        # Initialize hash
        h0 = uint32(0x6a09e667)
        h1 = uint32(0xbb67ae85)
        h2 = uint32(0x3c6ef372)
        h3 = uint32(0xa54ff53a)
        h4 = uint32(0x510e527f)
        h5 = uint32(0x9b05688c)
        h6 = uint32(0x1f83d9ab)
        h7 = uint32(0x5be0cd19)

        # Message schedule
        w = cuda.local.array(64, dtype=uint32)

        for i in range(16):
            w[i] = (uint32(msg_bytes[i*4]) << 24) | \
                   (uint32(msg_bytes[i*4+1]) << 16) | \
                   (uint32(msg_bytes[i*4+2]) << 8) | \
                   uint32(msg_bytes[i*4+3])

        for i in range(16, 64):
            w[i] = (cuda_gamma1(w[i-2]) + w[i-7] + cuda_gamma0(w[i-15]) + w[i-16]) & 0xFFFFFFFF

        # Working variables
        a, b, c, d, e, f, g, h = h0, h1, h2, h3, h4, h5, h6, h7

        # 64 rounds
        for i in range(64):
            t1 = (h + cuda_sigma1(e) + cuda_ch(e, f, g) + K[i] + w[i]) & 0xFFFFFFFF
            t2 = (cuda_sigma0(a) + cuda_maj(a, b, c)) & 0xFFFFFFFF
            h = g
            g = f
            f = e
            e = (d + t1) & 0xFFFFFFFF
            d = c
            c = b
            b = a
            a = (t1 + t2) & 0xFFFFFFFF

        # Output hash (as 8 uint32)
        hashes_out[idx, 0] = (h0 + a) & 0xFFFFFFFF
        hashes_out[idx, 1] = (h1 + b) & 0xFFFFFFFF
        hashes_out[idx, 2] = (h2 + c) & 0xFFFFFFFF
        hashes_out[idx, 3] = (h3 + d) & 0xFFFFFFFF
        hashes_out[idx, 4] = (h4 + e) & 0xFFFFFFFF
        hashes_out[idx, 5] = (h5 + f) & 0xFFFFFFFF
        hashes_out[idx, 6] = (h6 + g) & 0xFFFFFFFF
        hashes_out[idx, 7] = (h7 + h) & 0xFFFFFFFF

    @cuda.jit
    def cuda_compare_hashes(hashes, results, n):
        """Compare all hash pairs on GPU"""
        idx = cuda.grid(1)

        total_pairs = n * (n - 1) // 2
        if idx >= total_pairs:
            return

        # Convert linear index to pair (i, j)
        i = int((-1 + (1 + 8 * idx) ** 0.5) / 2)
        j = idx - i * (i + 1) // 2
        i, j = j, i + 1

        # Count equal bits via XOR
        equal_bits = 0
        for k in range(8):
            xor = hashes[i, k] ^ hashes[j, k]
            # Popcount
            while xor:
                equal_bits += 1 - (xor & 1)
                xor >>= 1

        # Store if better than current best
        cuda.atomic.max(results, 0, equal_bits)
        if equal_bits >= results[0]:
            results[1] = i
            results[2] = j
            results[0] = equal_bits

# ═══════════════════════════════════════════════════════════════════════════
# PARALLEL CPU WORKERS
# ═══════════════════════════════════════════════════════════════════════════

def worker_hash_batch(args):
    """Compute hashes for a batch of messages"""
    start, end = args
    results = {}
    for i in range(start, end):
        results[i] = sha256_bytes(i)
    return results

def worker_compare_batch(args):
    """Compare hash pairs in a batch"""
    hashes_dict, pairs = args

    best = (0, 0, 0)  # bits, m1, m2

    for m1, m2 in pairs:
        if m1 in hashes_dict and m2 in hashes_dict:
            bits = count_equal_bits_bytes(hashes_dict[m1], hashes_dict[m2])
            if bits > best[0]:
                best = (bits, m1, m2)

    return best

def worker_prefix_search(args):
    """Search with prefix grouping"""
    start, end, prefix_len = args

    prefixes = defaultdict(list)

    for i in range(start, end):
        h = sha256_hex(i)
        prefixes[h[:prefix_len]].append((i, h))

    best = (0, 0, 0)

    for group in prefixes.values():
        if len(group) >= 2:
            for i in range(len(group)):
                for j in range(i + 1, len(group)):
                    m1, h1 = group[i]
                    m2, h2 = group[j]
                    bits = count_equal_bits(h1, h2)
                    if bits > best[0]:
                        best = (bits, m1, m2)

    return best

def worker_neighborhood(args):
    """Search in neighborhood"""
    center1, center2, radius, samples, seed = args

    import random
    random.seed(seed)

    best = (0, 0, 0)

    points1 = [max(1, center1 + random.randint(-radius, radius)) for _ in range(samples)]
    points2 = [max(1, center2 + random.randint(-radius, radius)) for _ in range(samples)]

    hashes1 = {p: sha256_bytes(p) for p in set(points1)}
    hashes2 = {p: sha256_bytes(p) for p in set(points2)}

    for p1, h1 in hashes1.items():
        for p2, h2 in hashes2.items():
            if p1 != p2:
                bits = count_equal_bits_bytes(h1, h2)
                if bits > best[0]:
                    best = (bits, p1, p2)

    return best

def worker_random_search(args):
    """Random pair search"""
    n_pairs, msg_range, seed = args

    import random
    random.seed(seed)

    best = (0, 0, 0)

    for _ in range(n_pairs):
        m1 = random.randint(1, msg_range)
        m2 = random.randint(1, msg_range)

        if m1 != m2:
            h1 = sha256_bytes(m1)
            h2 = sha256_bytes(m2)
            bits = count_equal_bits_bytes(h1, h2)

            if bits > best[0]:
                best = (bits, m1, m2)

    return best

# ═══════════════════════════════════════════════════════════════════════════
# MAIN HUNTER CLASS
# ═══════════════════════════════════════════════════════════════════════════

class GPUCollisionHunter:

    def __init__(self):
        self.best_bits = 174
        self.best_m1 = 88514
        self.best_m2 = 88551
        self.best_h1 = sha256_hex(88514)
        self.best_h2 = sha256_hex(88551)
        self.total_ops = 0
        self.improvements = 0

    def update(self, bits, m1, m2):
        if bits > self.best_bits and m1 != m2:
            self.best_bits = bits
            self.best_m1 = m1
            self.best_m2 = m2
            self.best_h1 = sha256_hex(m1)
            self.best_h2 = sha256_hex(m2)
            self.improvements += 1
            return True
        return False

    def build_dimensional_space(self, N=100000):
        """Build dimensional space (parallelized)"""

        print(f"\n{'='*70}")
        print(f"⚡ PHASE 1: BUILDING DIMENSIONAL SPACE ({BACKEND})")
        print(f"{'='*70}")
        print(f"   Generating {N:,} points...")

        y_axis = np.arange(1, N + 1, dtype=np.float64)

        # Parallel hash computation
        batch_size = N // NUM_CORES

        with mp.Pool(NUM_CORES) as pool:
            args = [(i * batch_size + 1, (i + 1) * batch_size + 1) for i in range(NUM_CORES)]
            if args[-1][1] < N + 1:
                args[-1] = (args[-1][0], N + 1)

            results = pool.map(worker_hash_batch, args)

        # Merge results
        all_hashes = {}
        for r in results:
            all_hashes.update(r)

        # Convert to x_axis
        x_axis_raw = np.array([
            int.from_bytes(all_hashes[i], 'big')
            for i in range(1, N + 1)
        ], dtype=np.float64)

        max_val = float(2**256)
        x_axis = x_axis_raw / max_val
        x_axis_scaled = x_axis * float(N)

        print(f"   ✓ Space built with {N:,} points")

        return x_axis_scaled, y_axis, all_hashes

    def calculate_linear_function(self, x_axis_scaled, y_axis):
        """Calculate Y = mX + b"""

        print(f"\n{'='*70}")
        print(f"📊 PHASE 2: CALCULATING LINEAR FUNCTION")
        print(f"{'='*70}")

        n = len(x_axis_scaled)
        sum_x = np.sum(x_axis_scaled)
        sum_y = np.sum(y_axis)
        sum_xy = np.sum(x_axis_scaled * y_axis)
        sum_x2 = np.sum(x_axis_scaled ** 2)

        slope = (n * sum_xy - sum_x * sum_y) / (n * sum_x2 - sum_x ** 2)
        intercept = (sum_y - slope * sum_x) / n

        print(f"   Y = {slope:.10f} · X + {intercept:.10f}")

        return {'slope': slope, 'intercept': intercept}

    def calculate_collision_point(self, params):
        """Calculate collision point X = Y = Z"""

        print(f"\n{'='*70}")
        print(f"🎯 PHASE 3: CALCULATING COLLISION POINT")
        print(f"{'='*70}")

        slope = params['slope']
        intercept = params['intercept']

        if abs(slope - 1.0) > 1e-10:
            X_collision = intercept / (1.0 - slope)
        else:
            X_collision = intercept

        print(f"   X = Y = Z = {X_collision:.10f}")
        print(f"   Base point = {int(X_collision)}")

        return int(X_collision)

    def parallel_prefix_search(self, N=500000):
        """Parallel prefix-based search"""

        print(f"\n{'='*70}")
        print(f"🔤 PHASE 4A: PARALLEL PREFIX SEARCH ({BACKEND})")
        print(f"{'='*70}")

        for prefix_len in [7, 6, 5]:
            print(f"\n   Prefix length: {prefix_len}...")

            batch_size = N // NUM_CORES

            with mp.Pool(NUM_CORES) as pool:
                args = [(i * batch_size + 1, (i + 1) * batch_size + 1, prefix_len)
                        for i in range(NUM_CORES)]
                results = pool.map(worker_prefix_search, args)

            for bits, m1, m2 in results:
                if self.update(bits, m1, m2):
                    print(f"   ✨ NEW! {bits}/256 bits - M1={m1}, M2={m2}")

            self.total_ops += N

        print(f"   → Best: {self.best_bits}/256 bits")

    def parallel_random_search(self, n_pairs=10000000):
        """Parallel random search"""

        print(f"\n{'='*70}")
        print(f"🎲 PHASE 4B: PARALLEL RANDOM SEARCH ({BACKEND})")
        print(f"{'='*70}")
        print(f"   Pairs: {n_pairs:,} | Cores: {NUM_CORES}")

        pairs_per_worker = n_pairs // NUM_CORES

        with mp.Pool(NUM_CORES) as pool:
            args = [(pairs_per_worker, 100_000_000, i * 12345) for i in range(NUM_CORES)]
            results = pool.map(worker_random_search, args)

        for bits, m1, m2 in results:
            if self.update(bits, m1, m2):
                print(f"   ✨ NEW! {bits}/256 bits - M1={m1}, M2={m2}")

        self.total_ops += n_pairs
        print(f"   → Best: {self.best_bits}/256 bits")

    def parallel_neighborhood_search(self, iterations=50):
        """Parallel neighborhood refinement"""

        print(f"\n{'='*70}")
        print(f"🎯 PHASE 4C: PARALLEL NEIGHBORHOOD SEARCH ({BACKEND})")
        print(f"{'='*70}")

        for i in range(iterations):
            radius = max(100, 10000 - i * 200)
            samples = 500

            with mp.Pool(NUM_CORES) as pool:
                args = [(self.best_m1, self.best_m2, radius, samples, i * 1000 + j)
                        for j in range(NUM_CORES)]
                results = pool.map(worker_neighborhood, args)

            for bits, m1, m2 in results:
                if self.update(bits, m1, m2):
                    print(f"   ✨ Iter {i}: NEW! {bits}/256 bits")

            self.total_ops += NUM_CORES * samples * samples

            if i % 10 == 0:
                print(f"   Iter {i}/{iterations} - Radius: {radius} - Best: {self.best_bits}/256")

    def intensive_local_search(self, radius=500):
        """Intensive local search around best pair"""

        print(f"\n{'='*70}")
        print(f"🔬 PHASE 4D: INTENSIVE LOCAL SEARCH")
        print(f"{'='*70}")
        print(f"   Center: ({self.best_m1}, {self.best_m2}) | Radius: ±{radius}")

        range1 = list(range(max(1, self.best_m1 - radius), self.best_m1 + radius + 1))
        range2 = list(range(max(1, self.best_m2 - radius), self.best_m2 + radius + 1))

        all_points = list(set(range1 + range2))
        print(f"   Points: {len(all_points):,}")

        # Parallel hash computation
        print("   Computing hashes...")
        hashes = {p: sha256_bytes(p) for p in all_points}

        # Compare all pairs
        print("   Comparing pairs...")
        total_pairs = len(all_points) * (len(all_points) - 1) // 2
        checked = 0

        for i in range(len(all_points)):
            for j in range(i + 1, len(all_points)):
                p1, p2 = all_points[i], all_points[j]
                bits = count_equal_bits_bytes(hashes[p1], hashes[p2])

                if self.update(bits, p1, p2):
                    print(f"   ✨ NEW! {bits}/256 bits")

                checked += 1

            if i % 200 == 0 and i > 0:
                pct = 100 * checked / total_pairs
                print(f"   Progress: {checked:,}/{total_pairs:,} ({pct:.1f}%)")

        self.total_ops += total_pairs
        print(f"   → Best: {self.best_bits}/256 bits")

    def run(self, N_space=100000, N_search=500000, mode="normal"):
        """Run full analysis"""

        configs = {
            "quick": {
                "space": 50000,
                "prefix": 100000,
                "random": 5_000_000,
                "neighborhood": 20,
                "intensive": 200
            },
            "normal": {
                "space": 100000,
                "prefix": 500000,
                "random": 20_000_000,
                "neighborhood": 50,
                "intensive": 500
            },
            "intensive": {
                "space": 200000,
                "prefix": 1000000,
                "random": 50_000_000,
                "neighborhood": 100,
                "intensive": 1000
            },
            "extreme": {
                "space": 500000,
                "prefix": 2000000,
                "random": 100_000_000,
                "neighborhood": 200,
                "intensive": 2000
            }
        }

        cfg = configs.get(mode, configs["normal"])
        start = time.time()

        print(f"\n{'='*70}")
        print(f"🚀 STARTING - Mode: {mode.upper()} | Backend: {BACKEND}")
        print(f"{'='*70}")

        # Phase 1-3: Dimensional space
        x_scaled, y_axis, hashes = self.build_dimensional_space(cfg["space"])
        params = self.calculate_linear_function(x_scaled, y_axis)
        base_point = self.calculate_collision_point(params)

        # Phase 4: Search
        self.parallel_prefix_search(cfg["prefix"])
        self.parallel_random_search(cfg["random"])
        self.parallel_neighborhood_search(cfg["neighborhood"])
        self.intensive_local_search(cfg["intensive"])

        elapsed = time.time() - start
        return elapsed, params

    def show_results(self, elapsed, params):
        """Display results"""

        pct = self.best_bits / 256 * 100
        bar = "█" * int(pct // 5) + "░" * (20 - int(pct // 5))

        status = "🎉 NEW RECORD!" if self.best_bits > 174 else "📊 Result"

        print(f"""

{'='*70}
🏆 FINAL RESULTS
{'='*70}

╔═══════════════════════════════════════════════════════════════════════════╗
║                                                                           ║
║   {status:^67} ║
║                                                                           ║
║   MATCHING BITS: {self.best_bits}/256 ({pct:.2f}%)
║   [{bar}]
║                                                                           ║
║   M1 = {self.best_m1}
║   M2 = {self.best_m2}
║                                                                           ║
║   SHA-256(M1) = {self.best_h1}
║   SHA-256(M2) = {self.best_h2}
║                                                                           ║
║   Backend: {BACKEND}
║   Time: {elapsed:.1f} seconds
║   Operations: {self.total_ops:,}
║   Improvements: {self.improvements}
║                                                                           ║
╚═══════════════════════════════════════════════════════════════════════════╝
        """)

        # Bit map
        print("📊 BIT MAP (█=match, ░=diff):\n")
        b1 = bin(int(self.best_h1, 16))[2:].zfill(256)
        b2 = bin(int(self.best_h2, 16))[2:].zfill(256)

        for row in range(4):
            line = "   "
            for col in range(64):
                idx = row * 64 + col
                line += "█" if b1[idx] == b2[idx] else "░"
            print(line)

        # Formula
        print(f"""
📐 DIMENSIONAL FORMULA:

   Y = {params['slope']:.10f} · X + {params['intercept']:.10f}
   Collision Point: X = b/(1-m) = {params['intercept']/(1-params['slope']):.6f}

✅ VERIFICATION:
   M1 ≠ M2: {'YES ✓' if self.best_m1 != self.best_m2 else 'NO'}
   SHA-256 (64 rounds): YES ✓
        """)

# ═══════════════════════════════════════════════════════════════════════════
# MAIN
# ═══════════════════════════════════════════════════════════════════════════

def main():
    show_banner()
    show_theory()

    print(f"""
{'='*70}
⚙️  CONFIGURATION
{'='*70}

   Backend: {BACKEND}
   CPU Cores: {NUM_CORES}

   1. Quick      (~3-5 min)   - Fast test
   2. Normal     (~10-20 min) - Standard search
   3. Intensive  (~30-60 min) - Deep search
   4. Extreme    (~2+ hours)  - Maximum effort
    """)

    try:
        choice = input("   Mode [1-4, default=2]: ").strip() or "2"
    except:
        choice = "2"

    modes = {"1": "quick", "2": "normal", "3": "intensive", "4": "extreme"}
    mode = modes.get(choice, "normal")

    print(f"\n   → Mode: {mode.upper()}")
    input("   Press ENTER to start...")

    hunter = GPUCollisionHunter()
    elapsed, params = hunter.run(mode=mode)
    hunter.show_results(elapsed, params)

    print(f"""
{'='*70}
   ✅ COMPLETED
   Author: Kaoru Aguilera Katayama
   SHA-256 Collision Finder - GPU Accelerated
   Backend: {BACKEND}
{'='*70}
    """)

if __name__ == "__main__":
    mp.freeze_support()

    try:
        main()
    except KeyboardInterrupt:
        print("\n\n   Interrupted. See you! 👋")
    except Exception as e:
        print(f"\n   Error: {e}")
        import traceback
        traceback.print_exc()

✅ CUDA GPU detected!

╔═══════════════════════════════════════════════════════════════════════════╗
║                                                                           ║
║   ███████╗██╗  ██╗ █████╗       ██████╗ ███████╗ ██████╗                  ║
║   ██╔════╝██║  ██║██╔══██╗      ╚════██╗██╔════╝██╔════╝                  ║
║   ███████╗███████║███████║█████╗ █████╔╝███████╗███████╗                  ║
║   ╚════██║██╔══██║██╔══██║╚════╝██╔═══╝ ╚════██║██╔═══██╗                 ║
║   ███████║██║  ██║██║  ██║      ███████╗███████║╚██████╔╝                 ║
║   ╚══════╝╚═╝  ╚═╝╚═╝  ╚═╝      ╚══════╝╚══════╝ ╚═════╝                  ║
║                                                                           ║
║              COLLISION FINDER - GPU ACCELERATED                           ║
║                  Dimensional Space Method                                 ║
║                                                                           ║
║   🔥 Backend: CUDA       | Cores: 2      

Process ForkPoolWorker-14:




   Interrupted. See you! 👋


In [9]:
# ╔═══════════════════════════════════════════════════════════════════════════╗
# ║                                                                           ║
# ║   SHA-256 COLLISION FINDER - GPU ACCELERATED                             ║
# ║   Dimensional Space Method - PURE                                        ║
# ║                                                                           ║
# ║   Author: Kaoru Aguilera Katayama                                        ║
# ║                                                                           ║
# ║   RECORD: 174/256 bits (67.97%) - M1=88514, M2=88551                     ║
# ║                                                                           ║
# ╚═══════════════════════════════════════════════════════════════════════════╝

import hashlib
import numpy as np
from collections import defaultdict
import time
import warnings
import multiprocessing as mp

warnings.filterwarnings('ignore')

NUM_CORES = mp.cpu_count()
POPCOUNT_TABLE = [bin(i).count('1') for i in range(256)]

# ═══════════════════════════════════════════════════════════════════════════
# BANNER
# ═══════════════════════════════════════════════════════════════════════════

def show_banner():
    print(f"""
╔═══════════════════════════════════════════════════════════════════════════╗
║                                                                           ║
║   ███████╗██╗  ██╗ █████╗       ██████╗ ███████╗ ██████╗                  ║
║   ██╔════╝██║  ██║██╔══██╗      ╚════██╗██╔════╝██╔════╝                  ║
║   ███████╗███████║███████║█████╗ █████╔╝███████╗███████╗                  ║
║   ╚════██║██╔══██║██╔══██║╚════╝██╔═══╝ ╚════██║██╔═══██╗                 ║
║   ███████║██║  ██║██║  ██║      ███████╗███████║╚██████╔╝                 ║
║   ╚══════╝╚═╝  ╚═╝╚═╝  ╚═╝      ╚══════╝╚══════╝ ╚═════╝                  ║
║                                                                           ║
║              COLLISION FINDER - PARALLEL                                  ║
║            Dimensional Space Method - PURE                                ║
║                                                                           ║
║   🔥 CPU Cores: {NUM_CORES}                                                    ║
║   🎯 Record: 174/256 bits (67.97%)                                        ║
║   🚀 Target: 256/256 bits                                                 ║
║                                                                           ║
║   Author: Kaoru Aguilera Katayama                                        ║
║                                                                           ║
╚═══════════════════════════════════════════════════════════════════════════╝
    """)

def show_theory():
    print("""
═══════════════════════════════════════════════════════════════════════════
                    DIMENSIONAL SPACE METHOD - THEORY
═══════════════════════════════════════════════════════════════════════════

OBSERVATION:
    Prime numbers are NOT linear in normal space (1,2,3,4,5...)
    BUT they ARE linear in their own space (2,3,5,7,11...)

EUREKA:
    Same applies to SHA-256!
    In normal space: chaotic
    In DIMENSIONAL space: LINEAR

SPACE DEFINITION:

    📐 X-Axis: SHA(1), SHA(2), SHA(3)...  (hash values)
    📐 Y-Axis: 1, 2, 3, 4, 5...           (message indices)
    📐 Z-Axis: Collisions                  (meeting point)

METHOD:
    1. Build dimensional space with N points
    2. Calculate linear function: Y = m·X + b
    3. Find collision point: X = Y = Z
    4. Solve: X = b/(1-m)
    5. Search collisions via prefix grouping
    6. Refine in neighborhood

NO BRUTE FORCE - Direct algebraic calculation!

═══════════════════════════════════════════════════════════════════════════
    """)

# ═══════════════════════════════════════════════════════════════════════════
# CORE FUNCTIONS
# ═══════════════════════════════════════════════════════════════════════════

def sha256_int(msg):
    return int(hashlib.sha256(str(msg).encode()).hexdigest(), 16)

def sha256_hex(msg):
    return hashlib.sha256(str(msg).encode()).hexdigest()

def sha256_bytes(msg):
    return hashlib.sha256(str(msg).encode()).digest()

def count_equal_bits(h1, h2):
    if isinstance(h1, str):
        xor = int(h1, 16) ^ int(h2, 16)
    else:
        xor = int.from_bytes(h1, 'big') ^ int.from_bytes(h2, 'big')
    return 256 - bin(xor).count('1')

def count_equal_bits_bytes(h1_bytes, h2_bytes):
    different = 0
    for a, b in zip(h1_bytes, h2_bytes):
        different += POPCOUNT_TABLE[a ^ b]
    return 256 - different

# ═══════════════════════════════════════════════════════════════════════════
# PARALLEL WORKERS
# ═══════════════════════════════════════════════════════════════════════════

def worker_build_space(args):
    """Compute hashes for a range of messages"""
    start, end = args
    results = []
    for i in range(start, end):
        h = sha256_int(i)
        results.append((i, h))
    return results

def worker_prefix_search(args):
    """Search with prefix grouping"""
    start, end, prefix_lengths = args

    prefixes = defaultdict(list)

    for i in range(start, end):
        h = sha256_hex(i)
        for length in prefix_lengths:
            prefixes[(length, h[:length])].append((i, h))

    best = (0, 0, 0)  # bits, m1, m2

    for (length, prefix), group in prefixes.items():
        if len(group) >= 2:
            for i in range(len(group)):
                for j in range(i + 1, len(group)):
                    m1, h1 = group[i]
                    m2, h2 = group[j]
                    bits = count_equal_bits(h1, h2)
                    if bits > best[0]:
                        best = (bits, m1, m2)

    return best

def worker_refine_neighborhood(args):
    """Refine search around a center point"""
    center1, center2, radius, step = args

    best = (0, 0, 0)

    for dx1 in range(-radius, radius + 1, step):
        for dx2 in range(-radius, radius + 1, step):
            m1 = center1 + dx1
            m2 = center2 + dx2

            if m1 > 0 and m2 > 0 and m1 != m2:
                h1 = sha256_bytes(m1)
                h2 = sha256_bytes(m2)
                bits = count_equal_bits_bytes(h1, h2)

                if bits > best[0]:
                    best = (bits, m1, m2)

    return best

def worker_collision_point_search(args):
    """Search around collision point"""
    base_point, radius, step = args

    prefixes = defaultdict(list)

    start = max(1, base_point - radius)
    end = base_point + radius

    for i in range(start, end, step):
        h = sha256_hex(i)
        for length in [4, 5, 6, 7]:
            prefixes[(length, h[:length])].append((i, h))

    best = (0, 0, 0)

    for (length, prefix), group in prefixes.items():
        if len(group) >= 2:
            for i in range(len(group)):
                for j in range(i + 1, len(group)):
                    m1, h1 = group[i]
                    m2, h2 = group[j]
                    bits = count_equal_bits(h1, h2)
                    if bits > best[0]:
                        best = (bits, m1, m2)

    return best

# ═══════════════════════════════════════════════════════════════════════════
# MAIN HUNTER CLASS
# ═══════════════════════════════════════════════════════════════════════════

class DimensionalSpaceHunter:

    def __init__(self):
        self.best_bits = 174
        self.best_m1 = 88514
        self.best_m2 = 88551
        self.best_h1 = sha256_hex(88514)
        self.best_h2 = sha256_hex(88551)
        self.improvements = 0
        self.params = None
        self.collision_point = None

    def update(self, bits, m1, m2):
        if bits > self.best_bits and m1 != m2:
            self.best_bits = bits
            self.best_m1 = m1
            self.best_m2 = m2
            self.best_h1 = sha256_hex(m1)
            self.best_h2 = sha256_hex(m2)
            self.improvements += 1
            return True
        return False

    def phase1_build_space(self, N=100000):
        """PHASE 1: Build dimensional space (parallel)"""

        print(f"\n{'='*70}")
        print(f"⚡ PHASE 1: BUILDING DIMENSIONAL SPACE")
        print(f"{'='*70}")
        print(f"   Generating {N:,} points using {NUM_CORES} cores...")

        # Parallel hash computation
        batch_size = N // NUM_CORES

        with mp.Pool(NUM_CORES) as pool:
            args = [(i * batch_size + 1, (i + 1) * batch_size + 1) for i in range(NUM_CORES)]
            # Fix last batch
            args[-1] = (args[-1][0], N + 1)

            results = pool.map(worker_build_space, args)

        # Merge and sort results
        all_points = []
        for batch in results:
            all_points.extend(batch)
        all_points.sort(key=lambda x: x[0])

        # Build axes
        y_axis = np.array([float(p[0]) for p in all_points])
        x_axis_raw = np.array([float(p[1]) for p in all_points])

        # Normalize
        max_val = float(2**256)
        x_axis = x_axis_raw / max_val
        x_axis_scaled = x_axis * float(N)

        print(f"   ✓ Space built with {N:,} points")

        return x_axis_scaled, y_axis

    def phase2_linear_function(self, x_axis_scaled, y_axis):
        """PHASE 2: Calculate Y = mX + b"""

        print(f"\n{'='*70}")
        print(f"📊 PHASE 2: CALCULATING LINEAR FUNCTION")
        print(f"{'='*70}")

        n = len(x_axis_scaled)
        sum_x = np.sum(x_axis_scaled)
        sum_y = np.sum(y_axis)
        sum_xy = np.sum(x_axis_scaled * y_axis)
        sum_x2 = np.sum(x_axis_scaled ** 2)

        slope = (n * sum_xy - sum_x * sum_y) / (n * sum_x2 - sum_x ** 2)
        intercept = (sum_y - slope * sum_x) / n

        y_pred = slope * x_axis_scaled + intercept
        ss_res = np.sum((y_axis - y_pred) ** 2)
        ss_tot = np.sum((y_axis - np.mean(y_axis)) ** 2)
        r_squared = 1 - (ss_res / ss_tot)

        self.params = {
            'slope': slope,
            'intercept': intercept,
            'r_squared': r_squared
        }

        print(f"""
   📈 LINEAR FUNCTION:

      Y = m · X + b

      m (slope)     = {slope:.15f}
      b (intercept) = {intercept:.15f}
      R²            = {r_squared:.15f}
        """)

        return self.params

    def phase3_collision_point(self):
        """PHASE 3: Calculate X = Y = Z"""

        print(f"\n{'='*70}")
        print(f"🎯 PHASE 3: CALCULATING COLLISION POINT")
        print(f"{'='*70}")

        slope = self.params['slope']
        intercept = self.params['intercept']

        print("""
   Solving: X = Y = Z

   Given Y = mX + b, if X = Y:
      X = mX + b
      X(1-m) = b
      X = b/(1-m)
        """)

        if abs(slope - 1.0) > 1e-10:
            X_collision = intercept / (1.0 - slope)
        else:
            X_collision = intercept

        self.collision_point = int(X_collision)

        print(f"""
   🔴 COLLISION POINT:

      X = Y = Z = {X_collision:.10f}
      Base point = {self.collision_point}
        """)

        return self.collision_point

    def phase4_prefix_search(self, N=500000):
        """PHASE 4: Parallel prefix grouping search"""

        print(f"\n{'='*70}")
        print(f"🔍 PHASE 4: PARALLEL PREFIX SEARCH")
        print(f"{'='*70}")
        print(f"   Searching {N:,} hashes using {NUM_CORES} cores...")

        batch_size = N // NUM_CORES
        prefix_lengths = [4, 5, 6, 7]

        with mp.Pool(NUM_CORES) as pool:
            args = [(i * batch_size + 1, (i + 1) * batch_size + 1, prefix_lengths)
                    for i in range(NUM_CORES)]
            args[-1] = (args[-1][0], N + 1, prefix_lengths)

            results = pool.map(worker_prefix_search, args)

        for bits, m1, m2 in results:
            if self.update(bits, m1, m2):
                print(f"   ✨ NEW! {bits}/256 bits - M1={m1}, M2={m2}")

        print(f"   → Best: {self.best_bits}/256 bits")

    def phase5_collision_point_search(self):
        """PHASE 5: Search around collision point"""

        print(f"\n{'='*70}")
        print(f"🎯 PHASE 5: COLLISION POINT NEIGHBORHOOD")
        print(f"{'='*70}")
        print(f"   Searching around base point: {self.collision_point}")

        radii = [1000, 5000, 10000, 50000]

        for radius in radii:
            print(f"\n   Radius: ±{radius}...")

            # Divide into chunks for parallel processing
            chunk_size = (2 * radius) // NUM_CORES

            with mp.Pool(NUM_CORES) as pool:
                args = [(self.collision_point - radius + i * chunk_size,
                         chunk_size, 1) for i in range(NUM_CORES)]
                results = pool.map(worker_collision_point_search, args)

            for bits, m1, m2 in results:
                if self.update(bits, m1, m2):
                    print(f"   ✨ NEW! {bits}/256 bits - M1={m1}, M2={m2}")

        print(f"\n   → Best: {self.best_bits}/256 bits")

    def phase6_refinement(self, iterations=50):
        """PHASE 6: Refine best pair"""

        print(f"\n{'='*70}")
        print(f"🔬 PHASE 6: REFINEMENT ({iterations} iterations)")
        print(f"{'='*70}")

        for i in range(iterations):
            radius = max(10, 1000 - i * 20)
            step = max(1, radius // 50)

            # Parallel refinement around current best
            with mp.Pool(NUM_CORES) as pool:
                # Each worker searches a quadrant
                args = [
                    (self.best_m1, self.best_m2, radius, step),
                    (self.best_m1 + radius//2, self.best_m2, radius, step),
                    (self.best_m1, self.best_m2 + radius//2, radius, step),
                    (self.best_m1 - radius//2, self.best_m2 - radius//2, radius, step),
                ]

                # Add more workers if available
                while len(args) < NUM_CORES:
                    args.append((self.best_m1 + (len(args) * 100),
                                self.best_m2 + (len(args) * 100), radius, step))

                results = pool.map(worker_refine_neighborhood, args[:NUM_CORES])

            for bits, m1, m2 in results:
                if self.update(bits, m1, m2):
                    print(f"   ✨ Iter {i}: NEW! {bits}/256 bits")

            if i % 10 == 0:
                print(f"   Iter {i}/{iterations} - Radius: {radius} - Best: {self.best_bits}/256")

        print(f"\n   → Best: {self.best_bits}/256 bits")

    def phase7_intensive(self, radius=500):
        """PHASE 7: Intensive local search"""

        print(f"\n{'='*70}")
        print(f"🔥 PHASE 7: INTENSIVE LOCAL SEARCH")
        print(f"{'='*70}")
        print(f"   Center: ({self.best_m1}, {self.best_m2}) | Radius: ±{radius}")

        # Generate all points
        range1 = list(range(max(1, self.best_m1 - radius), self.best_m1 + radius + 1))
        range2 = list(range(max(1, self.best_m2 - radius), self.best_m2 + radius + 1))

        all_points = list(set(range1 + range2))
        print(f"   Points: {len(all_points):,}")

        # Compute hashes
        print("   Computing hashes...")
        hashes = {p: sha256_bytes(p) for p in all_points}

        # Compare all pairs
        print("   Comparing pairs...")
        total_pairs = len(all_points) * (len(all_points) - 1) // 2
        checked = 0

        for i in range(len(all_points)):
            for j in range(i + 1, len(all_points)):
                p1, p2 = all_points[i], all_points[j]
                bits = count_equal_bits_bytes(hashes[p1], hashes[p2])

                if self.update(bits, p1, p2):
                    print(f"   ✨ NEW! {bits}/256 bits")

                checked += 1

            if i % 200 == 0 and i > 0:
                pct = 100 * checked / total_pairs
                print(f"   Progress: {checked:,}/{total_pairs:,} ({pct:.1f}%)")

        print(f"\n   → Best: {self.best_bits}/256 bits")

    def run(self, mode="normal"):
        """Run full dimensional space analysis"""

        configs = {
            "quick": {
                "space": 50000,
                "prefix": 100000,
                "refine_iters": 20,
                "intensive": 200
            },
            "normal": {
                "space": 100000,
                "prefix": 500000,
                "refine_iters": 50,
                "intensive": 500
            },
            "intensive": {
                "space": 200000,
                "prefix": 1000000,
                "refine_iters": 100,
                "intensive": 1000
            },
            "extreme": {
                "space": 500000,
                "prefix": 2000000,
                "refine_iters": 200,
                "intensive": 2000
            }
        }

        cfg = configs.get(mode, configs["normal"])
        start = time.time()

        print(f"\n{'='*70}")
        print(f"🚀 STARTING - Mode: {mode.upper()}")
        print(f"{'='*70}")

        # Phase 1: Build space
        x_scaled, y_axis = self.phase1_build_space(cfg["space"])

        # Phase 2: Linear function
        self.phase2_linear_function(x_scaled, y_axis)

        # Phase 3: Collision point
        self.phase3_collision_point()

        # Phase 4: Prefix search
        self.phase4_prefix_search(cfg["prefix"])

        # Phase 5: Collision point neighborhood
        self.phase5_collision_point_search()

        # Phase 6: Refinement
        self.phase6_refinement(cfg["refine_iters"])

        # Phase 7: Intensive
        self.phase7_intensive(cfg["intensive"])

        elapsed = time.time() - start
        return elapsed

    def show_results(self, elapsed):
        """Display results"""

        pct = self.best_bits / 256 * 100
        bar = "█" * int(pct // 5) + "░" * (20 - int(pct // 5))

        status = "🎉 NEW RECORD!" if self.best_bits > 174 else "📊 Result"

        print(f"""

{'='*70}
🏆 FINAL RESULTS
{'='*70}

╔═══════════════════════════════════════════════════════════════════════════╗
║                                                                           ║
║   {status:^67} ║
║                                                                           ║
║   MATCHING BITS: {self.best_bits}/256 ({pct:.2f}%)
║   [{bar}]
║                                                                           ║
║   M1 = {self.best_m1}
║   M2 = {self.best_m2}
║                                                                           ║
║   SHA-256(M1) = {self.best_h1}
║   SHA-256(M2) = {self.best_h2}
║                                                                           ║
║   Time: {elapsed:.1f} seconds
║   Improvements: {self.improvements}
║                                                                           ║
╚═══════════════════════════════════════════════════════════════════════════╝
        """)

        # Bit map
        print("📊 BIT MAP (█=match, ░=diff):\n")
        b1 = bin(int(self.best_h1, 16))[2:].zfill(256)
        b2 = bin(int(self.best_h2, 16))[2:].zfill(256)

        for row in range(4):
            line = "   "
            for col in range(64):
                idx = row * 64 + col
                line += "█" if b1[idx] == b2[idx] else "░"
            print(line)

        # Formula
        if self.params:
            slope = self.params['slope']
            intercept = self.params['intercept']
            r_squared = self.params['r_squared']

            print(f"""
📐 DIMENSIONAL FORMULA:

   Y = {slope:.15f} · X + {intercept:.15f}
   R² = {r_squared:.15f}

   Collision Point: X = b/(1-m) = {self.collision_point}

✅ VERIFICATION:
   M1 ≠ M2: {'YES ✓' if self.best_m1 != self.best_m2 else 'NO'}
   SHA-256 (64 rounds): YES ✓
            """)

# ═══════════════════════════════════════════════════════════════════════════
# MAIN
# ═══════════════════════════════════════════════════════════════════════════

def main():
    show_banner()
    show_theory()

    print(f"""
{'='*70}
⚙️  CONFIGURATION
{'='*70}

   CPU Cores: {NUM_CORES}
   Method: Dimensional Space (PURE)

   1. Quick      (~3-5 min)   - Fast test
   2. Normal     (~10-20 min) - Standard search
   3. Intensive  (~30-60 min) - Deep search
   4. Extreme    (~2+ hours)  - Maximum effort
    """)

    try:
        choice = input("   Mode [1-4, default=2]: ").strip() or "2"
    except:
        choice = "2"

    modes = {"1": "quick", "2": "normal", "3": "intensive", "4": "extreme"}
    mode = modes.get(choice, "normal")

    print(f"\n   → Mode: {mode.upper()}")
    input("   Press ENTER to start...")

    hunter = DimensionalSpaceHunter()
    elapsed = hunter.run(mode=mode)
    hunter.show_results(elapsed)

    print(f"""
{'='*70}
   ✅ COMPLETED
   Author: Kaoru Aguilera Katayama
   Method: Dimensional Space - PURE
{'='*70}
    """)

if __name__ == "__main__":
    mp.freeze_support()

    try:
        main()
    except KeyboardInterrupt:
        print("\n\n   Interrupted. See you! 👋")
    except Exception as e:
        print(f"\n   Error: {e}")
        import traceback
        traceback.print_exc()


╔═══════════════════════════════════════════════════════════════════════════╗
║                                                                           ║
║   ███████╗██╗  ██╗ █████╗       ██████╗ ███████╗ ██████╗                  ║
║   ██╔════╝██║  ██║██╔══██╗      ╚════██╗██╔════╝██╔════╝                  ║
║   ███████╗███████║███████║█████╗ █████╔╝███████╗███████╗                  ║
║   ╚════██║██╔══██║██╔══██║╚════╝██╔═══╝ ╚════██║██╔═══██╗                 ║
║   ███████║██║  ██║██║  ██║      ███████╗███████║╚██████╔╝                 ║
║   ╚══════╝╚═╝  ╚═╝╚═╝  ╚═╝      ╚══════╝╚══════╝ ╚═════╝                  ║
║                                                                           ║
║              COLLISION FINDER - PARALLEL                                  ║
║            Dimensional Space Method - PURE                                ║
║                                                                           ║
║   🔥 CPU Cores: 2                                             